<h1 style="font-size:50px; text-align:center">DCO Detections</h1>

In [47]:
import h5py as h5
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import seaborn as sns
import legwork
import os
import sys
sys.path.append("../src/")
from variations import variations
from galaxy import simulate_mw

In [48]:
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

params = {'legend.fontsize': fs,
         'axes.labelsize': fs,
         'xtick.labelsize':0.7*fs,
         'ytick.labelsize':0.7*fs}
plt.rcParams.update(params)

In [50]:
dco_colours = {"BHBH": plt.get_cmap("plasma")(0.2), 
               "BHNS": plt.get_cmap("plasma")(0.5),
               "NSNS": plt.get_cmap("plasma")(0.8)}

dco_types = ["BHBH", "BHNS", "NSNS"]
data_path = "../data/"
sim_folder = data_path + "simulation/"

total_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW.npy")

# Calculate number of detections

In [51]:
def get_detections(dco_type, MW_SIZE=200000, MW_MASS_FAC=1.0, t_obs=4*u.yr, only_frac=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))
    
    # go through each physics variation
#     for d in range(len(dco_types)):
    
    for d in range(len(dco_types)):
        for v in range(len(variations)):
            # open the proper output file
            fname = sim_folder + "{}_{}_all.h5".format(dco_types[d], variations[v]["file"])
            if os.path.isfile(fname):
                with h5.File(fname, "r") as f:
                    parameter = "n_detect"
                    if v in [7, 8, 9, 17, 19]:
                        parameter = "n_ten_year"
                    n_detect = f["simulation"].attrs[parameter].astype(np.int)
                    total_mw_weight = f["simulation"].attrs["total_MW_weight"]
                    full_data = f["simulation"][...].squeeze()
                
#                 circ_sources = legwork.source.Source(m_1=full_data["m_1"] * u.Msun, m_2=full_data["m_2"] * u.Msun,
#                                            dist=full_data["dist"] * u.kpc, ecc=np.zeros(len(full_data["m_1"])),
#                                            a=full_data["a_LISA"] * u.AU)
#                 circ_snr = circ_sources.get_snr()
                
#                 print(dco_types[d])
#                 true_rate = np.sum(full_data["weight"][full_data["snr"] >= 7]) / np.sum(total_mw_weight) * total_DCOs_in_MW[d][v] * MW_MASS_FAC
#                 circ_rate = np.sum(full_data["weight"][circ_snr >= 7]) / np.sum(total_mw_weight) * total_DCOs_in_MW[d][v] * MW_MASS_FAC
#                 print(true_rate, circ_rate, (true_rate - circ_rate) / true_rate)
#                 print()

                # go through the file and normalise the detections for all binaries with SNR > x
                cursor = 0
                detections_per_MW = np.zeros(len(n_detect))
                
                checks = np.zeros(len(n_detect))
                for i in range(len(n_detect)):
                    snr = full_data["snr"][cursor:cursor + n_detect[i]]
                    weights = full_data["weight"][cursor:cursor + n_detect[i]]
                    checks = np.sum(weights)**2 / np.sum(weights**2)
                    detections[d][v][i] = np.sum(weights[snr * np.sqrt(t_obs / (4 * u.yr)) > 7]) / total_mw_weight[i]
                    cursor += n_detect[i]
                if not only_frac:
                    detections[d][v] *= total_DCOs_in_MW[d][v] * MW_MASS_FAC

            # set to zero if no file exists (simulation will have crashed)
            else:
                detections[d][v] = 0
                
            print(detections[d][v])
    
    return detections

In [5]:
def get_detections_alt(dco_type, MW_SIZE=100000, MW_MASS_FAC=1.5, t_obs=4*u.yr, only_frac=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))
    
    fname = sim_folder + "{}_{}_all.h5".format(dco_types[dco_type], variations[0]["file"])
    if os.path.isfile(fname):
        with h5.File(fname, "r") as f:
            total_mw_weight = f["simulation"].attrs["total_MW_weight"].sum()
            full_data = f["simulation"][...].squeeze()
            
        factor = total_DCOs_in_MW[dco_type][0] * MW_MASS_FAC / total_mw_weight
        
        weights = full_data["weight"][full_data["snr"] > 7]
        avg = weights.sum() * factor
        print(avg)
        
    sample_rates = np.zeros(1000)
    for i in range(len(sample_rates)):
        sample = np.random.choice(weights, len(weights), replace=True)
        sample_rates[i] = sample.sum() * factor
        
    print(np.percentile(sample_rates, [5, 95]))
    
    sample_rates = np.zeros(1000)
    for i in range(len(sample_rates)):
        sample = np.random.choice(weights, 100, replace=True)
        sample_rates[i] = sample.sum() * factor * (len(weights) / 100)
        
    print(np.percentile(sample_rates, [5, 95]))

In [52]:
detections_4yr = get_detections(dco_types)

[128.75553669  63.06312425  75.57047455 ...  51.96995253  77.33089551
  57.66246165]
[30.31776695 60.03412938 44.81810477 ... 64.45213089 60.70161075
 82.1707407 ]
[38.59636526 40.32541847 45.48455549 ... 46.3389492  36.60031676
 57.00963637]
[53.7095463  71.32829094 68.77145818 ... 25.7907076  63.93962085
 59.72147587]
[49.46868757 59.25840177 22.14038803 ... 58.27997258 70.71667549
 37.92373272]
[178.29296486 175.53584697 200.23458072 ...  60.41353385  95.19651672
 207.5978456 ]
[31.10562327 28.26714841 25.75328044 ... 26.61596304 24.72652701
 13.55701753]
[ 59.84984434  41.75670287  97.69051265 ... 110.31679008  53.62054762
 113.96540859]
[ 54.06738343  51.32082327 105.54099479 ...  61.93694525  99.14078286
  66.33141567]
[0. 0. 0. ... 0. 0. 0.]
[151.32410424  55.46310353  87.15391573 ... 215.53566226 127.30733283
 168.2665776 ]
[38.90199851 62.33482076 57.71517767 ... 44.01386382 59.1332984
 75.82676538]
[137.3338922  183.33190218  55.99302763 ...  83.1171024   49.42977511
  75.669

<ipython-input-51-955773ba0689>:40: RuntimeWarning: invalid value encountered in double_scalars
  checks = np.sum(weights)**2 / np.sum(weights**2)


[24.67012874 14.77025099 15.59923964 ... 23.43960807 25.05968262
 19.06218668]
[10.63582133  5.063011   11.88443939 ...  8.71506708  5.92582297
 11.75163939]
[ 9.55728124  4.53381028  6.93382537 ... 11.21813173 11.54231163
  5.81899393]
[10.47405414  6.08083969 14.72876013 ...  4.90230787  3.39729728
  4.8508405 ]


In [6]:
detections_10yr = get_detections(dco_types, t_obs=10*u.yr)

In [7]:
detections_only_fractions = get_detections(dco_types, only_frac=True)

In [54]:
np.save("../data/detections_4yr", detections_4yr)
np.save("../data/detections_10yr", detections_10yr)
np.save("../data/detections_only_fractions", detections_only_fractions)

NameError: name 'detections_10yr' is not defined

# What if we assumed everything was circular?

In [38]:
fid_sources = [None, None, None]
total_mw_weights = [None, None, None]
for i in range(len(dco_types)):
    with h5.File(sim_folder + "{}_massTransferEfficiencyFixed_0_5_all.h5".format(dco_types[i]), "r") as f:
        full_data = f["simulation"][...].squeeze()
        snr_mask = full_data["snr"] > 7
        
        data = full_data[snr_mask]
        
        total_mw_weights[i] = f["simulation"].attrs["total_MW_weight"].sum()
        
    fid_sources[i] = legwork.source.Source(m_1=data["m_1"] * u.Msun, m_2=data["m_2"] * u.Msun,
                                           dist=data["dist"] * u.kpc, ecc=data["e_LISA"],
                                           a=data["a_LISA"] * u.AU)
    fid_sources[i].weight = data["weight"]
    fid_sources[i].zams_mask = np.logical_and(data["m_1_ZAMS"] >= 20, data["m_1_ZAMS"] <= 120)

In [40]:
for i in range(len(dco_types)):
    snr_true = fid_sources[i].get_snr()
    new_source = fid_sources[i]
    new_source.ecc = np.zeros(new_source.n_sources)
    snr_circ = new_source.get_snr()
    
    full_mask = np.logical_and(fid_sources[i].zams_mask, snr_circ > 7)
    
    print(np.sum(new_source.weight[full_mask]) / total_mw_weights[i] * total_DCOs_in_MW[i][2])

40.60807861360013
10.22671235981745
0.1015822905330661
